In [1]:
import cv2
import numpy as np
from pdf2image import convert_from_path

In [ ]:

# Convert the PDF to images
pdf_path = 'floor_plan.pdf'  # Replace with your PDF file path
pages = convert_from_path(pdf_path, dpi=300)  # Higher DPI for better resolution

# Process each page
for i, page in enumerate(pages):
    # Convert PIL image to OpenCV format
    img = np.array(page)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detection to highlight edges
    edges = cv2.Canny(blurred, threshold1=50, threshold2=150, apertureSize=3)

    # Use dilation to strengthen wall edges
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(edges, kernel, iterations=2)

    # Hough Transform to detect strong lines that could represent walls
    lines = cv2.HoughLinesP(dilated, rho=1, theta=np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

    # Create an empty image to draw wall lines
    wall_lines = np.zeros_like(img)

    # Draw the detected lines onto the empty image
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(wall_lines, (x1, y1), (x2, y2), (255, 255, 255), 2)

    # Convert wall_lines to grayscale to make it easier to view
    wall_lines_gray = cv2.cvtColor(wall_lines, cv2.COLOR_BGR2GRAY)

    # Save the result for each page
    output_path = f"extracted_wall_lines_page_{i + 1}.png"
    cv2.imwrite(output_path, wall_lines_gray)

    # Display images (comment out if running in non-GUI environment)
    cv2.imshow("Original", img)
    cv2.imshow("Edges", edges)
    cv2.imshow("Wall Lines", wall_lines_gray)
    cv2.waitKey(0)

cv2.destroyAllWindows()
